In [93]:
import pandas as pd
import numpy as np
import random
import os
import sys
inputname=sys.argv[1]
temp_fasta=sys.argv[2]
outputname=sys.argv[3]
print "start program"
# read fasta file and generating it into array sequences
def read_fasta_file(path):
    '''
    used for load fasta data and transformd into numpy.array format
    '''
    fh=open(path)
    seq=[]
    for line in fh:
        if line.startswith('>'):
            continue
        else:
            seq.append(line.replace('\r','').replace('\n',''))
    fh.close()
    matrix_data=np.array([list(e) for e in seq])
    return matrix_data




def generating_sequences_function(sequences,seed):
    random.seed(seed)
    generating_sequences_temp=[]
    for index,line in enumerate(sequences):
        generating_sequences_temp.append(">name"+str(index+1)+"\n")
        random.shuffle(line)
        line="".join(list(line))
        generating_sequences_temp.append(line+"\n")
    return generating_sequences_temp

#minimum free energy
def read_MFE_from_log(MFE_file_path):
    MFE_line_temp=[]
    with open(MFE_file_path) as f:
        lines=f.readlines()
        for index in xrange(len(lines)):
            if (index+1)%3==0:
                MFE_line_temp.append(float(lines[index][(len(lines[2])-7):len(lines[2])-2]))
    return MFE_line_temp
        

from sklearn import preprocessing
MFE_all=[]
original_sequences=read_fasta_file(inputname)
os.system("RNAfold < "+inputname+" > "+outputname)
MFE_line=read_MFE_from_log(outputname)
MFE_all.append(MFE_line)
for echo_num in xrange(1,101):
    original_sequences=read_fasta_file(inputname)
    generating_sequences=generating_sequences_function(original_sequences,echo_num)
    with open(temp_fasta,'w') as f:
        f.writelines(generating_sequences)
    os.system("RNAfold < "+temp_fasta+" > "+outputname)
    MFE_line=read_MFE_from_log(outputname)
    MFE_all.append(MFE_line)
    print "echo ",echo_num,"is done"
Standerscaler=preprocessing.StandardScaler()
MFE_all=Standerscaler.fit_transform(MFE_all)
MFE_all=np.array(MFE_all).T[0]
print np.array(MFE_all).shape   
pd.DataFrame(MFE_all).to_csv(outputname, header=None, index=None)    
print "end program"
    

IndexError: list index out of range